In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.cluster import KMeans
from skimage import io
from glob import glob
from tqdm import tqdm

In [ ]:
train_path='dataset1/train'

In [ ]:
train_images=[]
train_masks=[]
def load_images(path):
    temp_img,temp_mask=[],[]
    images=glob(os.path.join(path,'*.jpg'))
    for i in tqdm(images):
        i=cv2.imread(i)
        img=i[:,:256]
        msk=i[:,256:] 
        temp_img.append(img)
        temp_mask.append(msk)
    return temp_img,temp_mask

train_images,train_masks=load_images(train_path)

In [ ]:
train_images = np.array(train_images)
train_masks = np.array(train_masks)
print(train_images.shape, train_masks.shape)

In [ ]:
color_array = np.random.choice(range(256), 3*1000).reshape(-1,3)
print(color_array.shape)

In [ ]:
def classify_segment(image_array, colors=15):    
    labels = []
    kmeans = KMeans(n_clusters=colors, random_state=0)
    color_array = np.random.choice(range(256), 3*1000).reshape(-1,3)
    color_array = shuffle(color_array, random_state=0)[:1000]
    kmeans.fit(color_array)
    for image in tqdm(image_array):               
        cluster_labels = kmeans.predict(image.reshape(-1,3))
        cluster_labels = cluster_labels.reshape((256, 256))
        converted_image = np.zeros((256, 256, colors), dtype=np.uint8)
        for i in range(colors):
            converted_image[:, :, i] = (cluster_labels == i)
        #composite_image = converted_image.astype(np.uint8)
        labels.append(converted_image)
    return labels

In [ ]:
train_masks = classify_segment(train_masks)

In [ ]:
plt.imshow(train_images[87])

In [ ]:
plt.imshow(np.argmax(train_masks[87], axis=-1))

In [ ]:
train_masks = np.array(train_masks)

In [ ]:
from sklearn.model_selection import train_test_split

trainx, testx, trainy, testy = train_test_split(train_images,train_masks, test_size=0.25, random_state=40)
print(trainx.shape, testx.shape, trainy.shape, testy.shape)

In [ ]:
trainy = trainy.astype(np.uint8)
testy = testy.astype(np.uint8)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *

In [ ]:
def U_net():
        # Build U-Net model
        inputs = Input((256, 256, 3))
        s = Lambda(lambda x: x / 255) (inputs)
        
        c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (s)
        c1 = Dropout(0.1) (c1)
        c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c1)
        p1 = MaxPooling2D((2, 2)) (c1)

        c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
        c2 = Dropout(0.1) (c2)
        c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c2)
        p2 = MaxPooling2D((2, 2)) (c2)

        c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
        c3 = Dropout(0.2) (c3)
        c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c3)
        p3 = MaxPooling2D((2, 2)) (c3)

        c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p3)
        c4 = Dropout(0.2) (c4)
        c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c4)
        p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

        c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p4)
        c5 = Dropout(0.3) (c5)
        c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c5)

        u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
        u6 = concatenate([u6, c4])
        c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u6)
        c6 = Dropout(0.2) (c6)
        c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c6)

        u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
        u7 = concatenate([u7, c3])
        c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u7)
        c7 = Dropout(0.2) (c7)
        c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c7)

        u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
        u8 = concatenate([u8, c2])
        c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u8)
        c8 = Dropout(0.1) (c8)
        c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c8)

        u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
        u9 = concatenate([u9, c1], axis=3)
        c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u9)
        c9 = Dropout(0.1) (c9)
        c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c9)

        outputs = Conv2D(15, (1, 1), activation='softmax') (c9)

        model = Model(inputs=[inputs], outputs=[outputs])
        model.summary()
        return model

In [ ]:
model = U_net()

In [ ]:
plt.imshow(trainx[1])

In [ ]:
plt.imshow(np.argmax(trainy[1], axis=-1))

In [ ]:
opt=Adam(learning_rate=1e-3)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks=[
    ModelCheckpoint("unet.h5", verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    CSVLogger("check.csv"),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False)
]

In [ ]:
model.fit(trainx, trainy, batch_size=8, epochs = 100, validation_data = (testx, testy), callbacks=callbacks)

In [ ]:
out = model.predict(np.expand_dims(testx[1],axis=0))[0]
out = (out * 255).astype(np.uint8)
plt.imshow(np.argmax(out, axis=-1))

In [ ]:
plt.imshow(np.argmax(testy[1], axis=-1))

In [ ]:
out = model.predict(np.expand_dims(trainx[87],axis=0))[0]
out = (out * 255).astype(np.uint8)
plt.imshow(np.argmax(out, axis=-1))